In [16]:
import requests
import pandas as pd
import time

# Kakao API 키
KAKAO_API_KEY = "My_key"

In [13]:
# Kakao API 키
def get_lat_lng(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                coords = result['documents'][0]['address']
                return coords['y'], coords['x']
            else:
                return None, None
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None, None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None, None

# 데이터 불러오기
file_path = "Data/산업단지별_매핑된_업종명.csv"
df = pd.read_csv(file_path)

# 위도와 경도 컬럼 추가
df['위도'] = None
df['경도'] = None

for idx, row in df.iterrows():
    lat, lng = get_lat_lng(row['공장대표주소(지번, 도로명 주소)'])
    df.at[idx, '위도'] = lat
    df.at[idx, '경도'] = lng
    print(f"Processed {idx + 1}/{len(df)}: {row['공장대표주소(지번, 도로명 주소)']} -> {lat}, {lng}")
    time.sleep(0.5)  # API 요청 속도 제한 대비

# 결과 저장
df.to_csv("산업단지별_위경도.csv", index=False, encoding='utf-8-sig')
df

Processed 1/1940: 경기도 화성시 방교동 846-6번지 -> 37.1615847388083, 127.086287138001
Processed 2/1940: 경기도 화성시 방교동 875-3번지 -> 37.1804000821656, 127.086244571301
Processed 3/1940: 경기도 화성시 방교동 832-4번지 -> 37.1698682358147, 127.086792990102
Processed 4/1940: 경기도 화성시 방교동 842-5 -> 37.1659242053485, 127.089718069094
Processed 5/1940: 경기도 화성시 방교동 836-4번지 -> 37.1709290564277, 127.090634898106
Processed 6/1940: 경기도 화성시 방교동 838-11번지 -> 37.1689566163709, 127.091725271081
Processed 7/1940: 경기도 화성시 방교동 800-1번지 -> 37.1674440353041, 127.085050781568
Processed 8/1940: 경기도 화성시 방교동 839-5 -> 37.1661472215319, 127.086191105946
Processed 9/1940: 경기도 화성시 동탄산단9길 23-6 -> 37.1756041287383, 127.090942149173
Processed 10/1940: 경기도 화성시 방교동 832-1번지 -> 37.1720082125046, 127.087326454191
Processed 11/1940: 경기도 화성시 방교동 835-3번지 -> 37.1703813471764, 127.088672812738
Processed 12/1940: 경기도 화성시 방교동 832-7 -> 37.1697359162038, 127.087588326375
Processed 13/1940: 경기도 화성시 방교동 832-6 -> 37.1689769199671, 127.087393675391
Processed 14/19

,단지명,회사명,"공장대표주소(지번, 도로명 주소)",매핑된 업종명,위도,경도
0,화성동탄일반산업단지,(주)21세기,경기도 화성시 방교동 846-6번지,전기전자,37.1615847388083,127.086287138001
1,화성동탄일반산업단지,(주)가온솔루션,경기도 화성시 방교동 875-3번지,전기전자,37.1804000821656,127.086244571301
2,화성동탄일반산업단지,(주)경진티알엠,경기도 화성시 방교동 832-4번지,기타,37.1698682358147,127.086792990102
3,화성동탄일반산업단지,(주)고성엔지니어링,경기도 화성시 방교동 842-5,전기전자,37.1659242053485,127.089718069094
4,화성동탄일반산업단지,(주)고송이엔지,경기도 화성시 방교동 836-4번지,전기전자,37.1709290564277,127.090634898106
...,...,...,...,...,...,...
1935,화성동탄도시첨단산업단지,"혜종-524,525",경기도 화성시 영천동 204-3 5층 524호,기타,None,None
1936,화성동탄도시첨단산업단지,화원,경기도 화성시 영천동 204-3번지 3층 304,기타,None,None
1937,화성동탄도시첨단산업단지,후렉스산업,경기도 화성시 영천동 193-2번지 5층 513,기타,None,None
1938,화성동탄도시첨단산업단지,휴먼테크늄,경기도 화성시 영천동 265-2번지 2층 211,기타,None,None


In [44]:
df = pd.read_csv("Data/산업단지별_위경도.csv", index_col = 0)
df_null = df[df['위도'].isnull()][['회사명']]

In [47]:
df_null = df_null.reset_index()

In [53]:
import pandas as pd
import requests
import time

# 카카오 API 키

def get_lat_lon(company_name):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": company_name, "size": 10}  # 최대 10개의 결과를 가져옴

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        documents = response.json().get("documents", [])

        # 경기도 화성시에 위치한 회사 필터링
        candidates = [doc for doc in documents if "경기 화성시" in doc.get("address_name", "")]
        
        if len(candidates) == 1:
            # 유일한 회사라면 좌표 반환
            return float(candidates[0]["y"]), float(candidates[0]["x"])
        elif len(candidates) > 1:
            # 여러 개일 경우 None으로 처리
            return None, None
        else:
            # 화성시에 없는 경우
            return None, None
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None, None

# 데이터프레임 생성 예제
# df_null = pd.DataFrame({"회사명": ["회사A", "회사B", "회사C"]})

# 위도, 경도 컬럼 추가
df_null["위도"] = None
df_null["경도"] = None

# 회사명으로 위도, 경도 찾기
for idx, row in df_null.iterrows():
    company_name = row["회사명"]
    lat, lon = get_lat_lon(company_name)
    df_null.at[idx, "위도"] = lat
    df_null.at[idx,# 중복되는 (위도, 경도) 조합의 개수 계산
duplicate_count = df.duplicated(subset=['위도', '경도']).sum()

# 중복 그룹별 개수 확인
duplicate_groups = df[df.duplicated(subset=['위도', '경도'], keep=False)]
grouped_counts = duplicate_groups.groupby(['위도', '경도']).size().reset_index(name='중복 개수')

# 결과 출력
print(f"중복되는 (위도, 경도) 조합의 총 개수: {duplicate_count}")
print("\n중복된 (위도, 경도) 조합 목록:")
print(grouped_counts)
 "경도"] = lon
    print(f"{company_name}: 위도={lat}, 경도={lon}")
    time.sleep(0.5)  # API 요청 간 딜레이 (필수)

# 결과 확인
print(df_null)

(주)디아이씨: 위도=37.17187086915568, 경도=127.0917679779071
(주)티베라: 위도=None, 경도=None
넥스타테크놀로지(주): 위도=None, 경도=None
이화다이아몬드공업(주): 위도=37.165387396585, 경도=127.083752801554
지에이에셋코리아(주): 위도=None, 경도=None
(주)동우파워텍: 위도=None, 경도=None
(주)에이스정공: 위도=None, 경도=None
(주)유진이엔티: 위도=37.18055138061, 경도=126.789393836181
(주)한성GT: 위도=None, 경도=None
금풍산업: 위도=37.1833907620019, 경도=126.793066232412
세원펀칭: 위도=37.1773821069422, 경도=126.784731707135
한스(주): 위도=None, 경도=None
(주)네오유디유: 위도=None, 경도=None
(주)선우피엔이: 위도=None, 경도=None
(주)유진솔루션: 위도=37.1214060000707, 경도=126.981831879545
부산재활용사촌복지공장: 위도=None, 경도=None
신성정밀: 위도=None, 경도=None
(주)대유켐: 위도=None, 경도=None
(주)에이치케이한국트레이드: 위도=None, 경도=None
(주)지디티: 위도=37.187795835391995, 경도=126.67795333551521
금성호이스트크레인(주): 위도=None, 경도=None
나성산업: 위도=None, 경도=None
대진ENG: 위도=None, 경도=None
명진정밀공업: 위도=37.1875401929776, 경도=126.674092628852
비에스텍: 위도=37.2216723923746, 경도=127.070127512619
삼성의료고무(주): 위도=37.190456723857565, 경도=126.67239859088345
새참식품: 위도=37.18536844249682, 경도=126.6675273052915
서전: 위도=37.2170

In [56]:
df

,회사명,"공장대표주소(지번, 도로명 주소)",매핑된 업종명,위도,경도
단지명,,,,,
화성동탄일반산업단지,(주)21세기,경기도 화성시 방교동 846-6번지,전기전자,37.161585,127.086287
화성동탄일반산업단지,(주)가온솔루션,경기도 화성시 방교동 875-3번지,전기전자,37.180400,127.086245
화성동탄일반산업단지,(주)경진티알엠,경기도 화성시 방교동 832-4번지,기타,37.169868,127.086793
화성동탄일반산업단지,(주)고성엔지니어링,경기도 화성시 방교동 842-5,전기전자,37.165924,127.089718
화성동탄일반산업단지,(주)고송이엔지,경기도 화성시 방교동 836-4번지,전기전자,37.170929,127.090635
...,...,...,...,...,...
화성동탄도시첨단산업단지,"혜종-524,525",경기도 화성시 영천동 204-3 5층 524호,기타,NaN,NaN
화성동탄도시첨단산업단지,화원,경기도 화성시 영천동 204-3번지 3층 304,기타,NaN,NaN
화성동탄도시첨단산업단지,후렉스산업,경기도 화성시 영천동 193-2번지 5층 513,기타,NaN,NaN


In [58]:
df_null.set_index("단지명", inplace=True)

In [64]:
# 회사명이 동일한 경우 df의 위도, 경도 값을 df_null의 값으로 업데이트
for idx, row in df_null.iterrows():
    company_name = row["회사명"]
    # df에서 동일 회사명 필터링
    mask = df["회사명"] == company_name

    # 위도와 경도 값이 None이 아닌 경우에만 업데이트
    if pd.notna(row["위도"]) and pd.notna(row["경도"]):
        df.loc[mask, "위도"] = row["위도"]
        df.loc[mask, "경도"] = row["경도"]
        print(f"업데이트: {company_name} -> 위도={row['위도']}, 경도={row['경도']}")

업데이트: (주)디아이씨 -> 위도=37.17187086915568, 경도=127.0917679779071
업데이트: 이화다이아몬드공업(주) -> 위도=37.165387396585, 경도=127.083752801554
업데이트: (주)유진이엔티 -> 위도=37.18055138061, 경도=126.789393836181
업데이트: 금풍산업 -> 위도=37.1833907620019, 경도=126.793066232412
업데이트: 세원펀칭 -> 위도=37.1773821069422, 경도=126.784731707135
업데이트: (주)유진솔루션 -> 위도=37.1214060000707, 경도=126.981831879545
업데이트: (주)지디티 -> 위도=37.187795835391995, 경도=126.67795333551521
업데이트: 명진정밀공업 -> 위도=37.1875401929776, 경도=126.674092628852
업데이트: 비에스텍 -> 위도=37.2216723923746, 경도=127.070127512619
업데이트: 삼성의료고무(주) -> 위도=37.190456723857565, 경도=126.67239859088345
업데이트: 새참식품 -> 위도=37.18536844249682, 경도=126.6675273052915
업데이트: 서전 -> 위도=37.2170340406283, 경도=126.683003809127
업데이트: 켐맥스(주)  -> 위도=37.18045051443993, 경도=126.67664557611664
업데이트: 파도화학 -> 위도=37.1875162556509, 경도=126.679521087795
업데이트: (주)HS하이테크 -> 위도=37.14513404337312, 경도=127.02969759596702
업데이트: (주)네스텍코리아 -> 위도=37.146116275197905, 경도=127.02939181526564
업데이트: (주)대한엔지니어링  -> 위도=37.1422279893209, 경도=127.026459383728


In [77]:
df_filter = df[df['위도'].isnull() & ((df['매핑된 업종명'] == '전기전자') | (df['매핑된 업종명'] == '위험물등'))]

In [78]:
df_filter

,회사명,"공장대표주소(지번, 도로명 주소)",매핑된 업종명,위도,경도
단지명,,,,,
화성동탄일반산업단지,(주)티베라,경기도 화성시 동탄면 0번지 동탄일반산업단지 7-3블럭,전기전자,NaN,NaN
화성마도일반산업단지,(주)동우파워텍,경기도 화성시 마도면 쌍송리 마도지방산업단지 10블럭 4롯트,전기전자,NaN,NaN
화성발안일반산업단지,부산재활용사촌복지공장,경기도 화성시 향남읍 구문천리 905번지,전기전자,NaN,NaN
화성전곡해양일반산업단지,(주)대유켐,경기도 화성시 서신면 전곡리 630번지,위험물등,NaN,NaN
화성전곡해양일반산업단지,나성산업,경기도 화성시 서신면 장외리 95-26번지,위험물등,NaN,NaN
...,...,...,...,...,...
화성동탄도시첨단산업단지,오성시스템,경기도 화성시 영천동 265-2 10층 1005호,전기전자,NaN,NaN
화성동탄도시첨단산업단지,우도닷컴(주),"경기도 화성시 영천동 265-2번지 8층 811,812",전기전자,NaN,NaN
화성동탄도시첨단산업단지,"윤여춘-825,826호",경기도 화성시 영천동 204-3번지 8층 825,전기전자,NaN,NaN


In [81]:
# 공장대표주소(지번, 도로명 주소) 컬럼명을 변수로 저장
address_col = "공장대표주소(지번, 도로명 주소)"

# df_filter의 공장대표주소 목록을 집합으로 변환 (검색 속도 향상)
filter_addresses = set(df_filter[address_col])

# df에서 해당 주소를 가지는 행 삭제
df = df[~df[address_col].isin(filter_addresses)]

In [84]:
df = df[~df['위도'].isnull()]

In [90]:
df[((df['매핑된 업종명'] == '전기전자') | (df['매핑된 업종명'] == '위험물등'))].to_csv("위험물 회사 주소.csv", encoding='utf-8-sig')

In [94]:
df

,회사명,"공장대표주소(지번, 도로명 주소)",매핑된 업종명,위도,경도
단지명,,,,,
화성동탄일반산업단지,(주)21세기,경기도 화성시 방교동 846-6번지,전기전자,37.161585,127.086287
화성동탄일반산업단지,(주)가온솔루션,경기도 화성시 방교동 875-3번지,전기전자,37.180400,127.086245
화성동탄일반산업단지,(주)경진티알엠,경기도 화성시 방교동 832-4번지,기타,37.169868,127.086793
화성동탄일반산업단지,(주)고성엔지니어링,경기도 화성시 방교동 842-5,전기전자,37.165924,127.089718
화성동탄일반산업단지,(주)고송이엔지,경기도 화성시 방교동 836-4번지,전기전자,37.170929,127.090635
...,...,...,...,...,...
화성동탄도시첨단산업단지,하나산업(주),경기도 화성시 영천동 180-3,기타,37.215378,127.089697
화성동탄도시첨단산업단지,하마마츠포토닉스코리아(주),영천동 343-1,전기전자,33.248167,126.589178
화성동탄도시첨단산업단지,한국화학융합시험연구원 화성,경기도 화성시 영천동 824-4 1층,기타,37.214623,127.091047


In [96]:
df.reset_index()

,단지명,회사명,"공장대표주소(지번, 도로명 주소)",매핑된 업종명,위도,경도
0,화성동탄일반산업단지,(주)21세기,경기도 화성시 방교동 846-6번지,전기전자,37.161585,127.086287
1,화성동탄일반산업단지,(주)가온솔루션,경기도 화성시 방교동 875-3번지,전기전자,37.180400,127.086245
2,화성동탄일반산업단지,(주)경진티알엠,경기도 화성시 방교동 832-4번지,기타,37.169868,127.086793
3,화성동탄일반산업단지,(주)고성엔지니어링,경기도 화성시 방교동 842-5,전기전자,37.165924,127.089718
4,화성동탄일반산업단지,(주)고송이엔지,경기도 화성시 방교동 836-4번지,전기전자,37.170929,127.090635
...,...,...,...,...,...,...
1669,화성동탄도시첨단산업단지,하나산업(주),경기도 화성시 영천동 180-3,기타,37.215378,127.089697
1670,화성동탄도시첨단산업단지,하마마츠포토닉스코리아(주),영천동 343-1,전기전자,33.248167,126.589178
1671,화성동탄도시첨단산업단지,한국화학융합시험연구원 화성,경기도 화성시 영천동 824-4 1층,기타,37.214623,127.091047
1672,화성동탄도시첨단산업단지,현중테크,경기도 화성시 영천동 193-2번지 7층 705,기타,37.207288,126.865057
